In [22]:
import numpy as np
import utils
import os
import tensorflow as tf

In [95]:
RULE = 'spouse'
TRACE_LENGTH = 1

data = np.load(os.path.join('..','data','royalty.npz'))

triples,traces,nopred,entities,relations = utils.get_data(data,RULE)

NUM_ENTITIES = len(entities)
NUM_RELATIONS = len(relations)

ent2idx = dict(zip(entities, range(NUM_ENTITIES)))
rel2idx = dict(zip(relations, range(NUM_RELATIONS)))

idx2ent = dict(zip(range(NUM_ENTITIES),entities))
idx2rel = dict(zip(range(NUM_RELATIONS),relations))

In [3]:
explaine_data = np.load(os.path.join('..','data','preds','explaine_'+RULE+'_preds.npz'),allow_pickle=True)

In [4]:
true_triples = triples[explaine_data['test_idx']]
true_traces = traces[explaine_data['test_idx']][:,0:TRACE_LENGTH,:]

In [5]:
pred_traces = utils.idx2array(explaine_data['preds'],idx2ent,idx2rel)

In [31]:
adj_data = np.concatenate([triples,traces[:,0:TRACE_LENGTH,:].reshape(-1,3)],axis=0)

adj_data_sparse = utils.array2idx(adj_data,ent2idx,rel2idx)

In [32]:
adj_mats = utils.get_adj_mats(
    data=adj_data_sparse,
    num_entities=NUM_ENTITIES,
    num_relations=NUM_RELATIONS
)

In [101]:
#np.argwhere(tf.sparse.to_dense(adj_mats[4])[0,ent2idx["Al-Ma'mun>"],:].numpy())

In [61]:
unique = []

for list_ in pred_traces[:,:,1]:
    list_ = list(list_)
    if list_ not in unique:
        
        tup = tuple(list_)
        
        unique.append(tup)

In [62]:
true_traces[0][:,1]

array(['brother', 'parent'], dtype='<U76')

In [63]:
d = {}

for tup in unique:
        
    count = (tup == pred_traces[:,:,1]).all(axis=1).sum() 
    
    d[tup] = count

In [64]:
sorted(d.items(),key=lambda x:x[1],reverse=True)

[(('uncle', 'uncle'), 813),
 (('brother', 'uncle'), 312),
 (('uncle', 'brother'), 286),
 (('parent', 'uncle'), 198),
 (('uncle', 'parent'), 187),
 (('sister', 'uncle'), 170),
 (('uncle', 'sister'), 157),
 (('brother', 'brother'), 142),
 (('sister', 'brother'), 105),
 (('brother', 'parent'), 104),
 (('parent', 'brother'), 90),
 (('brother', 'sister'), 78),
 (('sister', 'sister'), 76),
 (('parent', 'parent'), 69),
 (('sister', 'parent'), 54),
 (('parent', 'sister'), 49)]

In [65]:
bp_idx = (pred_traces[:,:,1] == ['brother','parent']).all(axis=1)
pb_idx = (pred_traces[:,:,1] == ['parent','brother']).all(axis=1)

In [66]:
print(np.argwhere(bp_idx)[0])
print(np.argwhere(pb_idx)[0])

[39]
[72]


In [67]:
i = 72
true_triples[i]

array(['Al-Mutawakkil', 'uncle', "Al-Ma'mun>"], dtype='<U76')

In [68]:
true_traces[i]

array([["Al-Mu'tasim>", 'brother', "Al-Ma'mun>"],
       ['Al-Mutawakkil', 'parent', "Al-Mu'tasim>"]], dtype='<U76')

In [69]:
pred_traces[i]

array([["Al-Ma'mun>", 'parent', 'Harun_al-Rashid'],
       ["Al-Ma'mun>", 'brother', "Al-Mu'tasim>"]], dtype='<U76')

In [ ]:
#look at what % of triples use the uncle's 1st degree neighbors